In [7]:
from pynq import Overlay
overlay = Overlay('/home/xilinx/pynq/overlays/sha1_overlay/sha1_overlay.bit')

In [8]:
overlay?

In [9]:
add_ip = overlay.sha1_control_0
add_ip?

In [ ]:
#add_ip.write() #write address offset
#add_ip.read() #read address offset